# Preliminaries

Importing python packages and setting display parameters

In [ ]:
import numpy as np
import pandas as pd
import itertools as it
import scipy.stats as stats

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

import thesis_EAfunc as EAf
import thesis_visfunc as EAv

In [2]:
plt.style.use("bmh")
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

pd.set_option("display.latex.repr", True)
pd.set_option("display.latex.longtable", True)

# Fitness Landscape Definition

In [3]:
# Problem domain
x_min = -15
x_max = 15
y_min = -15
y_max = 15

# Known minimum
x_point = -1
y_point = -1

domain = (x_min, x_max, y_min, y_max)
point = (x_point, y_point)
img_size = (8.5, 4.25)

# Problem definition


def f(x, y):
    D = 2
    alpha = 1 / 8

    x = (x - 5) / 6
    y = (y - 5) / 6

    a = np.abs(x ** 2 + y ** 2 - D) ** (alpha * D)
    b = (0.5 * (x ** 2 + y ** 2) + (x + y)) / D

    return a + b + 0.5

In [4]:
# Testing the minimum
print(f(-1, -1))

0.0


In [ ]:
# Testing the function
print(f(-1.0, -1.0), f(-11.0, -9.0), f(11.0, 3.0), f(-6.0, 9.0))

# Setting up the experiment
64 Experiments
>L-> In each experiment, one set of parameters is used.
>>L-> 40 Replicates per experiment.
>>>L-> Each replicate is different due to randomness effects.

In [6]:
# starting seed
np.random.seed(42)

## Initializing data storage

In [7]:
mult_fit_cols = (
    ["exp"]
    + ["pop_s"]
    + ["b"]
    + ["mut_p"]
    + ["mut_s"]
    + ["p_sel"]
    + ["s_sel"]
    + ["run", "generation", "fitness_min", "fitness_max", "fitness_mean", "fitness_std"]
)
multi_fit = pd.DataFrame(columns=mult_fit_cols)
multi_fit = multi_fit.infer_objects()

## Parameter space for the experiment

### Initializing

In [ ]:
# Algorithm parameters
# Number of replicates, and generations per experiment
rep_n = 30
gen_f = 200

# Population size
pop_s = [10, 160]

# Parent subpopulation's selection method and size
par_selection = ["uniform", "tournament_k3"]
b = [0.5, 5]
par_s = [z * y for z in pop_s for y in b]

# Progeny subpopulation's size
prog_s = par_s

# Crossover Method
crossover = "uniform"
# Mutation method, probability and size
mutation = "random_all_gau_dis"
mut_p = [0.1, 0.9]
mut_s = [0.5, 5]

# New population selection method
sur_selection = ["fitness_proportional_selection", "uniform"]

### 2-Level Factors encoded values

In [14]:
inputs_labels = {
    "pop_s": "Population size",
    "b": "Progeny-to-population ratio",
    "mut_p": "Mutation Probability",
    "mut_s": "Mutation size",
    "p_sel": "Parent selection",
    "s_sel": "Survivor selection method",
}

dat = [
    ("pop_s", 10, 160, -1, 1, "Numerical"),
    ("b", 0.5, 5, -1, 1, "Numerical"),
    ("mut_p", 0.1, 0.9, -1, 1, "Numerical (<1)"),
    ("mut_s", 0.5, 5, -1, 1, "Numerical"),
    ("p_sel", "uniform", "tournament k3", -1, 1, "Categorical"),
    ("s_sel", "fitness proportional", "uniform", -1, 1, "Categorical"),
]

inputs_df = pd.DataFrame(
    dat,
    columns=[
        "Factor",
        "Value_low",
        "Value_high",
        "encoded_low",
        "encoded_high",
        "Variable type",
    ],
)
inputs_df = inputs_df.set_index(["Factor"])
inputs_df["Label"] = inputs_df.index.map(lambda z: inputs_labels[z])
inputs_df = inputs_df[
    ["Label", "Variable type", "Value_low", "Value_high", "encoded_low", "encoded_high"]
]

inputs_df

,Label,Variable type,Value_low,Value_high,encoded_low,encoded_high
Factor,,,,,,
pop_s,Population size,Numerical,10,160,-1,1
b,Progeny-to-population ratio,Numerical,0.5,5,-1,1
mut_p,Mutation Probability,Numerical (<1),0.1,0.9,-1,1
mut_s,Mutation size,Numerical,0.5,5,-1,1
p_sel,Parent selection,Categorical,uniform,tournament k3,-1,1
s_sel,Survivor selection method,Categorical,fitness proportional,uniform,-1,1


### Combining the 2-level Factors

We create a list with all the possible combinations of the 2-level factors

In [10]:
exp_par = list(it.product(pop_s, b, mut_p, mut_s, par_selection, sur_selection))
print('Cantidad de combinaciones de parametros en "exp_par" :' + str(len(exp_par)))
print()
print('Primera y última combinación de parametros en "exp_par":')
print("Secuencia (pop_s, b, mut_p, mut_s, p_sel, s_sel)")
print(exp_par[0])
print(exp_par[63])

Cantidad de combinaciones de parametros en "exp_par" :64

Primera y última combinación de parametros en "exp_par":
Secuencia (pop_s, b, mut_p, mut_s, p_sel, s_sel)
(10, 0.5, 0.1, 0.5, 'uniform', 'fitness_proportional_selection')
(160, 5, 0.9, 5, 'tournament_k3', 'uniform')


# Experiment execution

In [ ]:
%%time
exp_n = 1
for (zz, yy, xx, vv, uu, tt) in exp_par:
    sur_selection = tt
    par_selection = uu
    mut_s = vv
    mut_p = xx
    b = yy
    pop_s = zz
    prog_s = int(b * pop_s)
    par_s = prog_s

    fitness_res = EAf.EA_exp_only_fitness(
        rep_n,
        gen_f,
        f,
        domain,
        pop_s,
        par_s,
        prog_s,
        mut_p,
        mut_s,
        par_selection,
        crossover,
        mutation,
        sur_selection,
    )

    fitness_res.insert(0, "s_sel", tt)
    fitness_res.insert(0, "p_sel", uu)
    fitness_res.insert(0, "mut_s", vv)
    fitness_res.insert(0, "mut_p", xx)
    fitness_res.insert(0, "b", yy)
    fitness_res.insert(0, "pop_s", zz)
    fitness_res.insert(0, "exp", exp_n)
    multi_fit = multi_fit.append(fitness_res, ignore_index=True, sort=False)
    multi_fit = multi_fit.infer_objects()

    exp_n += 1

## Data storage

Writing the Data Frame to a pickle file

In [12]:
multi_fit.to_pickle("./Data/TEST_A_2L_FitData.gz", compression="gzip")

Reading the Data Frame from a pickle file

In [11]:
multi_fit = pd.read_pickle("./Data/TEST_A_2L_FitData.gz", compression="gzip")

In [12]:
multi_fit.tail()

,exp,pop_s,b,mut_p,mut_s,p_sel,s_sel,run,generation,fitness_min,fitness_max,fitness_mean,fitness_std,seed
385915,64,160,5.0,0.9,5.0,tournament_k3,uniform,29,196,0.363748,3.689288,1.520392,0.638087,5176.0
385916,64,160,5.0,0.9,5.0,tournament_k3,uniform,29,197,0.302477,4.404797,1.552364,0.715208,5176.0
385917,64,160,5.0,0.9,5.0,tournament_k3,uniform,29,198,0.302477,4.404797,1.514642,0.696536,5176.0
385918,64,160,5.0,0.9,5.0,tournament_k3,uniform,29,199,0.363748,4.654575,1.529129,0.664930,5176.0
385919,64,160,5.0,0.9,5.0,tournament_k3,uniform,29,200,0.296914,4.610679,1.624612,0.697311,5176.0


# Processing data for DOE Analysis

Storing the latest generation's population of each replicate

In [21]:
query = multi_fit["generation"] == gen_f
multi_final_fitness_res = multi_fit[query]

Reordering columns

In [21]:
multi_final_fitness_res = multi_final_fitness_res.drop(
    ["exp", "generation", "run", "seed"], axis=1
)
multi_final_fitness_res.columns = [
    "pop_s",
    "b",
    "mut_p",
    "mut_s",
    "p_sel",
    "s_sel",
    "f_min",
    "f_max",
    "f_mean",
    "f_std",
]
multi_final_fitness_res = multi_final_fitness_res[
    [
        "pop_s",
        "b",
        "mut_p",
        "mut_s",
        "p_sel",
        "s_sel",
        "f_min",
        "f_max",
        "f_mean",
        "f_std",
    ]
]
multi_final_fitness_res = multi_final_fitness_res.reset_index(drop=True)

Encoding values for DOE's Factors

In [ ]:
multi_final_fitness_res["pop_s"] = (
    multi_final_fitness_res["pop_s"].replace([10, 160], [-1, 1]).infer_objects()
)
multi_final_fitness_res["b"] = (
    multi_final_fitness_res["b"].replace([0.5, 5], [-1, 1]).infer_objects()
)
multi_final_fitness_res["mut_p"] = (
    multi_final_fitness_res["mut_p"].replace([0.1, 0.9], [-1, 1]).infer_objects()
)
multi_final_fitness_res["mut_s"] = (
    multi_final_fitness_res["mut_s"].replace([0.5, 5], [-1, 1]).infer_objects()
)
multi_final_fitness_res["p_sel"] = (
    multi_final_fitness_res["p_sel"]
    .replace(["uniform", "tournament_k3"], [-1, 1])
    .infer_objects()
)
multi_final_fitness_res["s_sel"] = (
    multi_final_fitness_res["s_sel"]
    .replace(["fitness_proportional_selection", "uniform"], [-1, 1])
    .infer_objects()
)

Exploring the Data Frame

In [22]:
multi_final_fitness_res.head()

,pop_s,b,mut_p,mut_s,p_sel,s_sel,f_min,f_max,f_mean,f_std
0,-1,-1.0,-1.0,-1.0,-1,-1,1.062947,1.062947,1.062947,0.0
1,-1,-1.0,-1.0,-1.0,-1,-1,0.212610,0.212610,0.212610,0.0
2,-1,-1.0,-1.0,-1.0,-1,-1,0.175744,0.175744,0.175744,0.0
3,-1,-1.0,-1.0,-1.0,-1,-1,0.363683,0.363683,0.363683,0.0
4,-1,-1.0,-1.0,-1.0,-1,-1,0.274916,0.274916,0.274916,0.0


In [24]:
multi_final_fitness_res.tail()

,pop_s,b,mut_p,mut_s,p_sel,s_sel,f_min,f_max,f_mean,f_std
1915,1,1.0,1.0,1.0,1,1,0.489430,3.635706,1.558799,0.612727
1916,1,1.0,1.0,1.0,1,1,0.412185,3.677844,1.562965,0.644010
1917,1,1.0,1.0,1.0,1,1,0.394899,3.866678,1.545745,0.568131
1918,1,1.0,1.0,1.0,1,1,0.583054,3.908122,1.601404,0.688775
1919,1,1.0,1.0,1.0,1,1,0.296914,4.610679,1.624612,0.697311


Storing the Factor Coding and DOE results Data Frames

In [25]:
inputs_df.to_pickle("./Data/TEST_A_DOE_code.gz", compression="gzip")
multi_final_fitness_res.to_pickle("./Data/TEST_A_DOE_data.gz", compression="gzip")